In [1]:
import sys 
import os 
import numpy as np 
import pandas as pd 
import pickle 

sys.path.append('/media/nghia/Nguyen NghiaW/RRS-2021/20210322_RRS/RRS/04_RRS/rrs_kit')

from rrs_kit.DataClass import DataPath
from rrs_kit.utils import (
  filter_sign,
  adjust_cbc,
  adjust_chem,
  get_target_df,
  get_merge_data,
  make_timestamp,
  make_sequence_data,
  make_2d_data
)


ModuleNotFoundError: No module named 'DataClass'

In [2]:
dp = DataPath()

valid_trn_sign = pd.read_csv(
    os.path.join(dp.valid_path, 'trn_abn_flowsheet.csv'),
    encoding = 'CP949'
)

valid_trn_sign_weird = pd.read_csv(
    os.path.join(dp.valid_path, 'trn_abn_flowsheet_weird.csv'),
    encoding = 'CP949'
)


valid_tst_sign = pd.read_csv(
    os.path.join(dp.valid_path, 'tst_abn_flowsheet.csv'),
    encoding = 'CP949'
)

valid_tst_sign_weird = pd.read_csv(
    os.path.join(dp.valid_path, 'tst_abn_flowsheet_weird.csv'),
    encoding = 'CP949'
)

valid_event = pd.read_csv(
    os.path.join(dp.valid_path, 'valid_event.csv')
)

valid_trn_nl_sign = pd.read_csv(
    os.path.join(dp.valid_path, 'trn_nl_flowsheet.csv')
)

valid_tst_nl_sign = pd.read_csv(
    os.path.join(dp.valid_path, 'tst_nl_flowsheet.csv')
)

valid_nl_sample = pd.read_csv(
    '/data/datasets/rrs-data/10yrs_samples/10yrs_trn_nl_sample.csv'
)



In [3]:
valid_event

,patient_id,gender,birthday,detection_time,event_time
0,CSYR0JDA,F,19460930,2015-11-06 21:00:00,2015-11-08 02:00:00
1,GLP4GNKW,M,19570907,2016-01-21 06:00:00,2016-01-23 19:00:00
2,JIBB4HVE,F,19470910,2015-10-07 10:00:00,2015-10-07 10:00:00
3,KKIQQZAT,M,19690617,2015-12-18 17:00:00,2015-12-19 17:00:00
4,YOYWOGBG,M,19460510,2014-10-16 08:00:00,2014-10-18 21:00:00
...,...,...,...,...,...
116,VIIDRILR,M,19390130,2014-07-16 04:00:00,2014-07-17 07:00:00
117,NV7WR2FI,M,19281201,2010-06-27 00:00:00,2010-06-28 22:00:00
118,GKM0DAJX,M,19330413,2010-02-21 01:00:00,2010-02-21 12:00:00
119,Q6DLCYZB,F,19271210,2010-08-11 20:00:00,2010-08-16 08:00:00


In [26]:
#valid_trn_sign[valid_trn_sign.patient.apply(lambda x: x[:8]) == 'ZZZZRQ96']

# Under construction

,patient,adjusted_time,HR,RR,SBP,SaO2,Temp,event_time,measurement_time
124373,ZZZZRQ96_20180513,2018-05-14 07:00:00,99.0,24.0,102.0,92.0,36.4,2018-05-14 14:28:00,2018-05-14 07:00:00
124374,ZZZZRQ96_20180513,2018-05-14 08:00:00,108.0,25.0,92.0,91.0,36.3,2018-05-14 14:28:00,2018-05-14 08:00:00
124375,ZZZZRQ96_20180513,2018-05-14 09:00:00,96.0,24.0,96.0,96.0,36.2,2018-05-14 14:28:00,2018-05-14 09:00:00
124376,ZZZZRQ96_20180513,2018-05-14 10:00:00,105.0,30.0,97.0,85.0,36.2,2018-05-14 14:28:00,2018-05-14 10:00:00
124377,ZZZZRQ96_20180513,2018-05-14 11:00:00,92.0,26.0,100.0,92.0,36.2,2018-05-14 14:28:00,2018-05-14 11:00:00
124378,ZZZZRQ96_20180513,2018-05-14 12:00:00,97.0,21.0,111.0,95.0,36.3,2018-05-14 14:28:00,2018-05-14 12:00:00
124379,ZZZZRQ96_20180513,2018-05-14 13:00:00,101.0,26.0,129.0,88.0,36.2,2018-05-14 14:28:00,2018-05-14 13:00:00
124380,ZZZZRQ96_20180513,2018-05-14 14:00:00,99.0,19.0,104.0,91.0,36.3,2018-05-14 14:28:00,2018-05-14 14:00:00


In [3]:
verbose = True
valid_sign = pd.concat([valid_trn_sign, valid_tst_sign])

valid_sign['patient_id'] = valid_sign['patient'].apply(lambda x: x[:8])

valid_sign = valid_sign[valid_sign['patient_id'].isin(valid_event['patient_id'])]

valid_sign = valid_sign.rename(columns={
'Temp': 'BT'
})  
valid_sign = valid_sign.drop(['event_time'], axis = 1)

valid_sign['measurement_time'] = pd.to_datetime(valid_sign['measurement_time'])

# filter vital sign 
valid_sign = filter_sign(valid_sign, 'patient_id', time = 'measurement_time')

In [143]:
valid_sign_nl = pd.concat([valid_trn_nl_sign, valid_tst_nl_sign])
valid_sign_nl['patient_id'] = valid_sign_nl['patient'].apply(lambda x: x[:8])
valid_sign_nl = valid_sign_nl[valid_sign_nl['patient_id'].isin(valid_nl_sample['patient_id'])]

valid_sign_nl = valid_sign_nl.rename(columns={
'Temp': 'BT'
})  

valid_sign_nl['measurement_time'] = pd.to_datetime(valid_sign_nl['measurement_time'])

# filter vital sign 
valid_sign_nl = filter_sign(valid_sign_nl, 'patient_id', time = 'measurement_time')


In [144]:
valid_nl_sample

,patient_id,gender,birthday,data_type,abn,Hgb,Platelet Count,WBC Count,event_time,measurement_time,...,Potassium,Sodium,Total Bilirubin,Total Calcium,Total Protein,HR,RR,SBP,SaO2,Temp
0,A1YFM8RK,M,19440428,train,False,11.5,223.0,5.8,NaN,2015-09-10 14:35:00,...,4.2,139.0,0.60,6.4,3.9,72.0,18.0,134.5,99.0,36.6
1,A466E7ZP,F,19590408,train,False,10.2,117.0,5.7,NaN,2012-12-07 15:54:00,...,3.6,142.0,0.57,10.8,7.1,99.0,23.0,90.0,100.0,36.4
2,A7OD133R,F,19340926,train,False,9.6,391.0,12.1,NaN,2014-08-08 12:02:00,...,5.1,131.0,0.40,7.7,6.2,90.0,25.0,187.0,100.0,36.0
3,A98YUML0,F,19531026,train,False,5.7,1.0,0.8,NaN,2015-11-27 18:06:00,...,3.6,139.0,0.60,8.6,5.2,98.0,14.0,103.5,98.0,36.6
4,AOSIKKFQ,M,19460602,train,False,14.8,93.0,5.1,NaN,2010-12-19 17:26:00,...,4.4,141.0,0.70,8.8,6.3,92.0,20.0,110.0,100.0,36.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,K97YOL9U,M,19551020,train,False,12.5,460.0,7.0,NaN,2013-09-12 15:37:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,GT8G124F,F,19590605,train,False,12.1,29.0,3.4,NaN,2012-10-01 23:53:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,FTF16KP4,M,19500327,train,False,11.4,227.0,4.2,NaN,2011-02-24 16:07:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,FQVO4GWE,F,19450111,train,False,8.2,385.0,5.8,NaN,2012-06-26 17:18:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:

  # join two tabels
  valid_data = pd.merge(valid_sign, valid_event, left_on = 'patient_id', right_on = 'patient_id', how = 'left')
  valid_data['event_time'] = pd.to_datetime(valid_data['event_time'])
  valid_data['adjusted_time'] = pd.to_datetime(valid_data['adjusted_time'])
  valid_data['detection_time'] = pd.to_datetime(valid_data['detection_time'])
  valid_data

  

,patient_id,measurement_time,patient,adjusted_time,HR,RR,SBP,SaO2,BT,gender,birthday,detection_time,event_time
0,ABG1W8Y6,2009-09-28 12:00:00,ABG1W8Y6_20090826,2009-09-28 12:00:00,103.0,24.0,130.0,98.0,37.1,F,19480515,2009-09-28 06:00:00,2009-09-28 21:00:00
1,ABG1W8Y6,2009-09-28 13:00:00,ABG1W8Y6_20090826,2009-09-28 13:00:00,91.0,19.0,139.0,97.0,36.2,F,19480515,2009-09-28 06:00:00,2009-09-28 21:00:00
2,ABG1W8Y6,2009-09-28 14:00:00,ABG1W8Y6_20090826,2009-09-28 14:00:00,84.0,18.0,141.0,97.0,37.2,F,19480515,2009-09-28 06:00:00,2009-09-28 21:00:00
3,ABG1W8Y6,2009-09-28 15:00:00,ABG1W8Y6_20090826,2009-09-28 15:00:00,84.0,20.0,153.5,98.0,36.3,F,19480515,2009-09-28 06:00:00,2009-09-28 21:00:00
4,ABG1W8Y6,2009-09-28 16:00:00,ABG1W8Y6_20090826,2009-09-28 16:00:00,85.0,21.0,150.5,98.0,37.0,F,19480515,2009-09-28 06:00:00,2009-09-28 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199095,ZPWAIZ82,2010-11-27 17:00:00,ZPWAIZ82_20101122,2010-11-27 17:00:00,114.0,35.0,133.0,96.0,36.5,M,19370320,2010-11-27 08:00:00,2010-11-27 22:00:00
199096,ZPWAIZ82,2010-11-27 18:00:00,ZPWAIZ82_20101122,2010-11-27 18:00:00,111.0,26.0,141.0,93.0,36.5,M,19370320,2010-11-27 08:00:00,2010-11-27 22:00:00
199097,ZPWAIZ82,2010-11-27 19:00:00,ZPWAIZ82_20101122,2010-11-27 19:00:00,105.0,27.0,128.0,94.0,36.3,M,19370320,2010-11-27 08:00:00,2010-11-27 22:00:00
199098,ZPWAIZ82,2010-11-27 20:00:00,ZPWAIZ82_20101122,2010-11-27 20:00:00,110.0,28.0,131.0,92.0,36.5,M,19370320,2010-11-27 08:00:00,2010-11-27 22:00:00


In [146]:
valid_nl = pd.merge(valid_sign_nl, valid_nl_sample[['patient_id', 'gender', 'birthday']], left_on = 'patient_id', right_on = 'patient_id', how = 'left')
valid_nl 


,patient_id,measurement_time,patient,adjusted_time,HR,RR,SBP,SaO2,BT,gender,birthday
0,A1YFM8RK,2016-12-14 17:00:00,A1YFM8RK_20161214,2016-12-14 17:00:00,72.0,18.0,134.5,99.0,36.6,M,19440428
1,A1YFM8RK,2016-12-14 18:00:00,A1YFM8RK_20161214,2016-12-14 18:00:00,62.0,18.0,135.0,99.0,37.0,M,19440428
2,A1YFM8RK,2016-12-14 19:00:00,A1YFM8RK_20161214,2016-12-14 19:00:00,62.0,22.0,123.0,100.0,37.1,M,19440428
3,A1YFM8RK,2016-12-14 20:00:00,A1YFM8RK_20161214,2016-12-14 20:00:00,64.0,19.0,118.0,100.0,36.9,M,19440428
4,A1YFM8RK,2016-12-14 21:00:00,A1YFM8RK_20161214,2016-12-14 21:00:00,69.0,14.0,120.0,100.0,36.6,M,19440428
...,...,...,...,...,...,...,...,...,...,...,...
476325,ZYEBEIQI,2014-06-17 20:00:00,ZYEBEIQI_20140610,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,M,19580720
476326,ZYEBEIQI,2014-06-17 21:00:00,ZYEBEIQI_20140610,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,M,19580720
476327,ZYEBEIQI,2014-06-17 22:00:00,ZYEBEIQI_20140610,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,M,19580720
476328,ZYEBEIQI,2014-06-17 23:00:00,ZYEBEIQI_20140610,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,M,19580720


In [147]:
# target
valid_data = get_target_df(valid_data)
#get_target_df(valid_data)
valid_data.target.value_counts()

1    43556
0    21573
Name: target, dtype: int64

In [148]:
valid_nl = get_target_df(valid_nl, False)
#get_target_df(valid_data)
valid_nl.target.value_counts()

0    476330
Name: target, dtype: int64

In [149]:
valid_data['gender'] = valid_data['gender'].astype('category').cat.codes
valid_data['TS'] = make_timestamp(valid_data, index = 'patient_id')
valid_data['birthday'] = pd.to_datetime(valid_data['birthday'])
valid_data['Age'] = (valid_data['event_time'] - valid_data['birthday']).astype('timedelta64[D]')
valid_data['Age'] = (valid_data['Age'] / (366)).round().astype(int)
valid_data = valid_data.drop(['patient', 'birthday'], axis = 1)
valid_data = valid_data.rename(columns = {
'gender': 'Gender'
})
print('-' * 50)
print('CBC data')
valid_blood_trn_cbc = pd.read_csv(
    os.path.join(dp.valid_path, 'trn_abn_cbc.csv'),
    encoding = 'CP949'
)
valid_blood_tst_cbc = pd.read_csv(
    os.path.join(dp.valid_path, 'tst_abn_cbc.csv'),
    encoding = 'CP949'
)

--------------------------------------------------
CBC data


In [150]:
valid_nl['gender'] = valid_nl['gender'].astype('category').cat.codes
valid_nl['TS'] = make_timestamp(valid_nl, index = 'patient_id')
valid_nl['birthday'] = pd.to_datetime(valid_nl['birthday'])
valid_nl['Age'] = (valid_nl['measurement_time'] - valid_nl['birthday']).astype('timedelta64[D]')
valid_nl['Age'] = (valid_nl['Age'] / (366)).round().astype(int)
valid_nl = valid_nl.drop(['patient', 'birthday'], axis = 1)
valid_nl = valid_nl.rename(columns = {
    'gender': 'Gender'
})


In [151]:
print('-' * 50)
print('CBC data')
valid_blood_trn_cbc_nl = pd.read_csv(
    os.path.join(dp.valid_path, 'trn_nl_cbc.csv'),
    encoding = 'CP949'
)
valid_blood_trn_cbc_nl

--------------------------------------------------
CBC data


,patient,adjusted_time,Hgb,Platelet Count,WBC Count,measurement_time
0,A01J9AP0_20150612,2015-06-12 20:00:00,7.7,12.0,4.0,2015-06-12 19:31:00
1,A01J9AP0_20150612,2015-06-12 21:00:00,7.7,12.0,4.0,2015-06-12 19:31:00
2,A01J9AP0_20150612,2015-06-12 22:00:00,7.7,12.0,4.0,2015-06-12 19:31:00
3,A01J9AP0_20150612,2015-06-12 23:00:00,7.7,12.0,4.0,2015-06-12 19:31:00
4,A01J9AP0_20150612,2015-06-13 00:00:00,7.7,12.0,4.0,2015-06-12 19:31:00
...,...,...,...,...,...,...
4764645,ZZZZRQ96_20180513,2018-05-20 02:00:00,9.4,82.0,16.5,2018-05-19 05:43:00
4764646,ZZZZRQ96_20180513,2018-05-20 03:00:00,9.4,82.0,16.5,2018-05-19 05:43:00
4764647,ZZZZRQ96_20180513,2018-05-20 04:00:00,9.4,82.0,16.5,2018-05-19 05:43:00
4764648,ZZZZRQ96_20180513,2018-05-20 05:00:00,9.4,82.0,16.5,2018-05-19 05:43:00


In [152]:
  valid_blood_cbc = pd.concat([valid_blood_trn_cbc, valid_blood_tst_cbc])
  valid_blood_cbc['patient_id'] = valid_blood_cbc['patient'].apply(lambda x: x[:8])
  valid_blood_cbc = valid_blood_cbc[valid_blood_cbc['patient_id'].isin(valid_event['patient_id'])]
  valid_blood_cbc = valid_blood_cbc.drop(['event_time', 'measurement_time', 'patient'], axis = 1)
  
  # filter cbc value
  valid_blood_cbc = adjust_cbc(valid_blood_cbc)
  valid_blood_cbc = valid_blood_cbc.rename(columns = {
    'Platelet Count': 'platelet'
  })
  ##########
  print('-' * 50)
  print('Chem data')
  valid_blood_trn_lab = pd.read_csv(
    os.path.join(dp.valid_path, 'trn_abn_chem.csv'),
    encoding = 'CP949'
  )
  valid_blood_tst_lab = pd.read_csv(
    os.path.join(dp.valid_path, 'tst_abn_chem.csv'),
    encoding = 'CP949'
  )

  valid_blood_chem = pd.concat([valid_blood_trn_lab, valid_blood_tst_lab])
  valid_blood_chem['patient_id'] = valid_blood_chem['patient'].apply(lambda x: x[:8])
  valid_blood_chem = valid_blood_chem[valid_blood_chem['patient_id'].isin(valid_event['patient_id'])]
  valid_blood_chem = valid_blood_chem.drop(['event_time', 'measurement_time', 'patient'], axis = 1)

  # adjust chem values
  valid_blood_chem = adjust_chem(valid_blood_chem)
  valid_blood_chem = valid_blood_chem.rename(columns = {
    'Total Bilirubin': 'Total bilirubin',
    'Total Protein': 'Total protein',
    'Total Calcium': 'Total calcium',
    'Alkaline Phosphatase' : 'Alkaline phosphatase',
    'Creatinine': 'Creatinin'
  })
  if verbose:
    print('Glucose freq: ')
    print( valid_blood_chem['Glucose'].value_counts())
    print('Sodium counts: ')
    print( valid_blood_chem['Sodium'].value_counts())
    print('Potassium counts: ')
    print( valid_blood_chem['Potassium'].value_counts())
    print('Chloride counts: ')
    print( valid_blood_chem['Chloride'].value_counts())

  # merge blood data

  valid_blood = pd.merge(valid_blood_cbc, valid_blood_chem, how = 'left', \
    on = ['patient_id', 'adjusted_time'])

  blood_properties = ['WBC Count', 'Platelet Count', 'Hgb','BUN', 'creatinin', 'Glucose', 
                  'Sodium', 'Potassium', 'Chloride', 'Total protein', 'Total bilirubin',
                  'Albumin', 'CRP','Total calcium', 'Lactate', 'Alkaline phosphatase',
                  'AST', 'ALT']

  #for p in blood_properties:
  #  valid_blood[p].fillna(round(valid_blood[p].mean(), 1), inplace=True)


--------------------------------------------------
Chem data
Glucose freq: 
2    8558
1    1010
0      73
Name: Glucose, dtype: int64
Sodium counts: 
1    5331
0    3648
2     662
Name: Sodium, dtype: int64
Potassium counts: 
1    6854
0    2136
2     651
Name: Potassium, dtype: int64
Chloride counts: 
1    6731
0    1870
2    1040
Name: Chloride, dtype: int64


In [153]:
valid_blood_cbc_nl = valid_blood_trn_cbc_nl#pd.concat([valid_blood_trn_cbc_nl])
valid_blood_cbc_nl['patient_id'] = valid_blood_cbc_nl['patient'].apply(lambda x: x[:8])
valid_blood_cbc_nl = valid_blood_cbc_nl[valid_blood_cbc_nl['patient_id'].isin(valid_nl_sample['patient_id'])]
#  valid_blood_cbc_nl = valid_blood_cbc_nl.drop(['measurement_time', 'patient'], axis = 1)
  
  # filter cbc value
valid_blood_cbc_nl = adjust_cbc(valid_blood_cbc_nl)
valid_blood_cbc_nl = valid_blood_cbc_nl.rename(columns = {
'Platelet Count': 'platelet'
})
valid_blood_cbc_nl 

,patient,adjusted_time,Hgb,platelet,WBC Count,measurement_time,patient_id
8712,A1YFM8RK_20150910,2015-09-10 15:00:00,11.5,223.0,1,2015-09-10 14:35:00,A1YFM8RK
8713,A1YFM8RK_20151001,2015-10-01 16:00:00,11.2,245.0,1,2015-10-01 15:56:00,A1YFM8RK
8714,A1YFM8RK_20151022,2015-10-22 16:00:00,10.7,189.0,0,2015-10-22 16:16:00,A1YFM8RK
8715,A1YFM8RK_20151022,2015-10-22 17:00:00,10.7,189.0,0,2015-10-22 16:16:00,A1YFM8RK
8716,A1YFM8RK_20151022,2015-10-22 18:00:00,10.7,189.0,0,2015-10-22 16:16:00,A1YFM8RK
...,...,...,...,...,...,...,...
4760584,ZZF3O2KG_20100629,2010-07-05 08:00:00,11.5,262.0,0,2010-07-03 10:58:00,ZZF3O2KG
4760585,ZZF3O2KG_20100629,2010-07-05 09:00:00,11.5,262.0,0,2010-07-03 10:58:00,ZZF3O2KG
4760586,ZZF3O2KG_20100629,2010-07-05 10:00:00,11.5,262.0,0,2010-07-03 10:58:00,ZZF3O2KG
4760587,ZZF3O2KG_20100629,2010-07-05 11:00:00,11.5,262.0,0,2010-07-03 10:58:00,ZZF3O2KG


In [154]:
##########
print('-' * 50)
print('Chem data')
valid_blood_trn_lab_nl = pd.read_csv(
    os.path.join(dp.valid_path, 'trn_nl_chem.csv'),
    encoding = 'CP949'
)

valid_blood_chem_nl = valid_blood_trn_lab_nl #pd.concat([valid_blood_trn_lab, valid_blood_tst_lab])
valid_blood_chem_nl['patient_id'] = valid_blood_chem_nl['patient'].apply(lambda x: x[:8])
valid_blood_chem_nl = valid_blood_chem_nl[valid_blood_chem_nl['patient_id'].isin(valid_nl_sample['patient_id'])]
valid_blood_chem_nl = valid_blood_chem_nl.drop(['measurement_time'], axis = 1)
valid_blood_chem_nl = valid_blood_chem_nl.rename(columns={
    'Chloride (염소)': 'Chloride',
    'CRP (Quantitation)-임상화학': 'CRP'
})


--------------------------------------------------
Chem data


In [155]:
# adjust chem values
valid_blood_chem_nl = adjust_chem(valid_blood_chem_nl)
valid_blood_chem_nl = valid_blood_chem_nl.rename(columns = {
'Total Bilirubin': 'Total bilirubin',
'Total Protein': 'Total protein',
'Total Calcium': 'Total calcium',
'Alkaline Phosphatase' : 'Alkaline phosphatase',
'Creatinine': 'Creatinin'
})

#for p in blood_properties:
#  valid_blood[p].fillna(round(valid_blood[p].mean(), 1), inplace=True)


In [156]:

valid_blood_nl = pd.merge(valid_blood_cbc_nl, valid_blood_chem_nl, how = 'left', \
on = ['patient_id', 'adjusted_time'])

blood_properties = ['WBC Count', 'Platelet Count', 'Hgb','BUN', 'Creatinin', 'Glucose', 
                'Sodium', 'Potassium', 'Chloride', 'Total protein', 'Total bilirubin',
                'Albumin', 'CRP','Total calcium', 'Lactate', 'Alkaline phosphatase',
                'AST', 'ALT']
valid_blood_nl

,patient_x,adjusted_time,Hgb,platelet,WBC Count,measurement_time,patient_id,patient_y,ALT,AST,...,CRP,Chloride,Creatinin,Glucose,Lactate,Potassium,Sodium,Total bilirubin,Total calcium,Total protein
0,A1YFM8RK_20150910,2015-09-10 15:00:00,11.5,223.0,1,2015-09-10 14:35:00,A1YFM8RK,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1YFM8RK_20151001,2015-10-01 16:00:00,11.2,245.0,1,2015-10-01 15:56:00,A1YFM8RK,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A1YFM8RK_20151022,2015-10-22 16:00:00,10.7,189.0,0,2015-10-22 16:16:00,A1YFM8RK,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A1YFM8RK_20151022,2015-10-22 17:00:00,10.7,189.0,0,2015-10-22 16:16:00,A1YFM8RK,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1YFM8RK_20151022,2015-10-22 18:00:00,10.7,189.0,0,2015-10-22 16:16:00,A1YFM8RK,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296287,ZZF3O2KG_20100629,2010-07-05 08:00:00,11.5,262.0,0,2010-07-03 10:58:00,ZZF3O2KG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296288,ZZF3O2KG_20100629,2010-07-05 09:00:00,11.5,262.0,0,2010-07-03 10:58:00,ZZF3O2KG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296289,ZZF3O2KG_20100629,2010-07-05 10:00:00,11.5,262.0,0,2010-07-03 10:58:00,ZZF3O2KG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296290,ZZF3O2KG_20100629,2010-07-05 11:00:00,11.5,262.0,0,2010-07-03 10:58:00,ZZF3O2KG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:

valid_blood['adjusted_time'] = pd.to_datetime(valid_blood['adjusted_time'])

# merge blood data into valid_data
valid_data = get_merge_data(valid_data, valid_blood)
valid_data = valid_data.rename(columns = {
    'patient_id': 'Patient'
})
valid_data['is_abn'] = 1

valid_data.to_csv(os.path.join(dp.valid_path, 'valid_merge.csv'), index = False)

window_len = 8

var_list = list(set(valid_data.columns)
                - set([ 'Patient', 'adjusted_time', 'detection_time', 'measurement_time', 'event_time', 'target', 'is_abn']) )


In [158]:
valid_data

,Patient,adjusted_time,measurement_time,HR,RR,SBP,SaO2,BT,Gender,detection_time,...,Chloride,Creatinin,Glucose,Lactate,Potassium,Sodium,Total bilirubin,Total calcium,Total protein,is_abn
0,ABG1W8Y6,2009-09-28 12:00:00,2009-09-28 12:00:00,103.0,24.0,130.0,98.0,37.1,0.0,2009-09-28 06:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,ABG1W8Y6,2009-09-28 13:00:00,2009-09-28 13:00:00,91.0,19.0,139.0,97.0,36.2,0.0,2009-09-28 06:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,ABG1W8Y6,2009-09-28 14:00:00,2009-09-28 14:00:00,84.0,18.0,141.0,97.0,37.2,0.0,2009-09-28 06:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,ABG1W8Y6,2009-09-28 15:00:00,2009-09-28 15:00:00,84.0,20.0,153.5,98.0,36.3,0.0,2009-09-28 06:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,ABG1W8Y6,2009-09-28 16:00:00,2009-09-28 16:00:00,85.0,21.0,150.5,98.0,37.0,0.0,2009-09-28 06:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16806,ZPWAIZ82,2010-11-27 17:00:00,2010-11-27 17:00:00,114.0,35.0,133.0,96.0,36.5,1.0,2010-11-27 08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16807,ZPWAIZ82,2010-11-27 18:00:00,2010-11-27 18:00:00,111.0,26.0,141.0,93.0,36.5,1.0,2010-11-27 08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16808,ZPWAIZ82,2010-11-27 19:00:00,2010-11-27 19:00:00,105.0,27.0,128.0,94.0,36.3,1.0,2010-11-27 08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16809,ZPWAIZ82,2010-11-27 20:00:00,2010-11-27 20:00:00,110.0,28.0,131.0,92.0,36.5,1.0,2010-11-27 08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [165]:

valid_blood_nl['adjusted_time'] = pd.to_datetime(valid_blood_nl['adjusted_time'])
valid_nl['adjusted_time'] = pd.to_datetime(valid_nl['adjusted_time'])

# merge blood data into valid_data
valid_data_nl = get_merge_data(valid_nl, valid_blood_nl)
 

,patient_id,adjusted_time,measurement_time_x,HR,RR,SBP,SaO2,BT,Gender,target,...,CRP,Chloride,Creatinin,Glucose,Lactate,Potassium,Sodium,Total bilirubin,Total calcium,Total protein
0,A1YFM8RK,2016-12-14 17:00:00,2016-12-14 17:00:00,72.0,18.0,134.5,99.0,36.6,1.0,0.0,...,0.14,2.0,1.6,2.0,2.2,1.0,1.0,0.6,6.4,3.9
1,A1YFM8RK,2016-12-14 18:00:00,2016-12-14 18:00:00,62.0,18.0,135.0,99.0,37.0,1.0,0.0,...,0.14,2.0,1.6,2.0,2.2,1.0,1.0,0.6,6.4,3.9
2,A1YFM8RK,2016-12-14 19:00:00,2016-12-14 19:00:00,62.0,22.0,123.0,100.0,37.1,1.0,0.0,...,0.14,2.0,1.6,2.0,2.2,1.0,1.0,0.6,6.4,3.9
3,A1YFM8RK,2016-12-14 20:00:00,2016-12-14 20:00:00,64.0,19.0,118.0,100.0,36.9,1.0,0.0,...,0.14,2.0,1.6,2.0,2.2,1.0,1.0,0.6,6.4,3.9
4,A1YFM8RK,2016-12-14 21:00:00,2016-12-14 21:00:00,69.0,14.0,120.0,100.0,36.6,1.0,0.0,...,0.14,2.0,1.6,2.0,2.2,1.0,1.0,0.6,6.4,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476325,ZYEBEIQI,2014-06-17 20:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476326,ZYEBEIQI,2014-06-17 21:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476327,ZYEBEIQI,2014-06-17 22:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476328,ZYEBEIQI,2014-06-17 23:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
valid_data_nl = valid_data_nl.drop(['measurement_time_y', 'patient_x', 'patient_y'], axis = 1)
valid_data_nl = valid_data_nl.rename(columns={
    'measurement_time_x': 'measurement_time'
})

In [173]:
valid_data_nl.columns

Index(['patient_id', 'adjusted_time', 'measurement_time', 'HR', 'RR', 'SBP',
       'SaO2', 'BT', 'Gender', 'target', 'TS', 'Age', 'Hgb', 'platelet',
       'WBC Count', 'ALT', 'AST', 'Albumin', 'Alkaline phosphatase', 'BUN',
       'CRP', 'Chloride', 'Creatinin', 'Glucose', 'Lactate', 'Potassium',
       'Sodium', 'Total bilirubin', 'Total calcium', 'Total protein'],
      dtype='object')

In [174]:
valid_data_nl = valid_data_nl.rename(columns = {
    'patient_id': 'Patient'
})
valid_data_nl['is_abn'] = 0

valid_data_nl.to_csv(os.path.join(dp.valid_path, 'valid_nl_merge.csv'), index = False)

window_len = 8

var_list = list(set(valid_data_nl.columns)
                - set([ 'Patient', 'adjusted_time', 'measurement_time' 'target', 'is_abn']) )

In [175]:
valid_nl

,patient_id,measurement_time,adjusted_time,HR,RR,SBP,SaO2,BT,Gender,target,TS,Age
0,A1YFM8RK,2016-12-14 17:00:00,2016-12-14 17:00:00,72.0,18.0,134.5,99.0,36.6,1,0,1,47
1,A1YFM8RK,2016-12-14 18:00:00,2016-12-14 18:00:00,62.0,18.0,135.0,99.0,37.0,1,0,2,47
2,A1YFM8RK,2016-12-14 19:00:00,2016-12-14 19:00:00,62.0,22.0,123.0,100.0,37.1,1,0,3,47
3,A1YFM8RK,2016-12-14 20:00:00,2016-12-14 20:00:00,64.0,19.0,118.0,100.0,36.9,1,0,4,47
4,A1YFM8RK,2016-12-14 21:00:00,2016-12-14 21:00:00,69.0,14.0,120.0,100.0,36.6,1,0,5,47
...,...,...,...,...,...,...,...,...,...,...,...,...
476325,ZYEBEIQI,2014-06-17 20:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1,0,168,44
476326,ZYEBEIQI,2014-06-17 21:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1,0,169,44
476327,ZYEBEIQI,2014-06-17 22:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1,0,170,44
476328,ZYEBEIQI,2014-06-17 23:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1,0,171,44


In [177]:
var_list = list(set(valid_data.columns)
                - set([ 'Patient', 'adjusted_time', 'detection_time', 'measurement_time', 'event_time', 'target', 'is_abn']) )
valid_seq_data = make_sequence_data(valid_data, window_len = window_len, var_list = var_list, index = 'Patient')



총 환자수: 111
Window 크기: 8
-------------------- Making Data --------------------
=> 100번 째 환자
-------------------- Done --------------------


In [193]:
valid_data_nl['detection_time']= np.nan #valid_data_nl['measurement_time']
valid_data_nl['event_time']= np.nan #valid_data_nl['measurement_time']


In [190]:
valid_data = valid_data[((valid_data.detection_time.dt.day - valid_data.measurement_time.dt.day) < 5)]


In [191]:
valid_data.target.value_counts()

0.0    7489
1.0    1527
Name: target, dtype: int64

In [192]:
valid_seq_data = make_sequence_data(valid_data, window_len = window_len, var_list = var_list, index = 'Patient')


총 환자수: 106
Window 크기: 8
-------------------- Making Data --------------------
=> 100번 째 환자
-------------------- Done --------------------


In [195]:
valid_data_nl = valid_data_nl.groupby('Patient', as_index = False).tail(36)

In [196]:
valid_data_nl

,Patient,adjusted_time,measurement_time,HR,RR,SBP,SaO2,BT,Gender,target,...,Glucose,Lactate,Potassium,Sodium,Total bilirubin,Total calcium,Total protein,is_abn,detection_time,event_time
20,A1YFM8RK,2016-12-15 13:00:00,2016-12-15 13:00:00,80.0,12.0,143.0,100.0,36.2,1.0,0.0,...,2.0,2.2,1.0,1.0,0.4,7.5,4.3,0,NaN,NaN
21,A1YFM8RK,2016-12-15 14:00:00,2016-12-15 14:00:00,87.0,11.0,136.0,100.0,36.3,1.0,0.0,...,2.0,2.2,1.0,1.0,0.4,7.5,4.3,0,NaN,NaN
22,A1YFM8RK,2016-12-15 15:00:00,2016-12-15 15:00:00,87.0,17.0,132.0,100.0,36.4,1.0,0.0,...,2.0,2.2,1.0,1.0,0.4,7.5,4.3,0,NaN,NaN
23,A1YFM8RK,2016-12-15 16:00:00,2016-12-15 16:00:00,84.0,16.0,139.0,100.0,36.4,1.0,0.0,...,2.0,2.2,1.0,1.0,0.4,7.5,4.3,0,NaN,NaN
24,A1YFM8RK,2016-12-15 17:00:00,2016-12-15 17:00:00,86.0,13.0,143.0,100.0,36.4,1.0,0.0,...,2.0,2.2,1.0,1.0,0.4,7.5,4.3,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476325,ZYEBEIQI,2014-06-17 20:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
476326,ZYEBEIQI,2014-06-17 21:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
476327,ZYEBEIQI,2014-06-17 22:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
476328,ZYEBEIQI,2014-06-17 23:00:00,2014-06-17 13:00:00,103.0,22.0,106.0,98.0,36.4,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [208]:
valid_seq_data_nl = make_sequence_data(valid_data_nl, window_len = window_len, var_list = var_list, index = 'Patient')

총 환자수: 597
Window 크기: 8
-------------------- Making Data --------------------
=> 100번 째 환자
==> 200번 째 환자
===> 300번 째 환자
====> 400번 째 환자
=====> 500번 째 환자
-------------------- Done --------------------


In [209]:
valid_seq = pd.concat([valid_seq_data, valid_seq_data_nl])

In [210]:
path = os.path.join(dp.valid_path, 'valid_seq.pickle')
with open(path, 'wb') as f:
    pickle.dump(valid_seq, f)


In [211]:
res_data = make_2d_data(valid_seq, var_list, \
    output_path = dp.valid_path, output_file = 'valid_final.csv')
  

column_name_multi:  ['HR-7', 'Chloride-7', 'RR-7', 'BUN-7', 'Glucose-7', 'SBP-7', 'Alkaline phosphatase-7', 'ALT-7', 'Total calcium-7', 'Sodium-7', 'platelet-7', 'BT-7', 'Albumin-7', 'SaO2-7', 'Creatinin-7', 'CRP-7', 'Gender-7', 'Potassium-7', 'Age-7', 'AST-7', 'Total bilirubin-7', 'TS-7', 'Hgb-7', 'Lactate-7', 'Total protein-7', 'WBC Count-7', 'HR-6', 'Chloride-6', 'RR-6', 'BUN-6', 'Glucose-6', 'SBP-6', 'Alkaline phosphatase-6', 'ALT-6', 'Total calcium-6', 'Sodium-6', 'platelet-6', 'BT-6', 'Albumin-6', 'SaO2-6', 'Creatinin-6', 'CRP-6', 'Gender-6', 'Potassium-6', 'Age-6', 'AST-6', 'Total bilirubin-6', 'TS-6', 'Hgb-6', 'Lactate-6', 'Total protein-6', 'WBC Count-6', 'HR-5', 'Chloride-5', 'RR-5', 'BUN-5', 'Glucose-5', 'SBP-5', 'Alkaline phosphatase-5', 'ALT-5', 'Total calcium-5', 'Sodium-5', 'platelet-5', 'BT-5', 'Albumin-5', 'SaO2-5', 'Creatinin-5', 'CRP-5', 'Gender-5', 'Potassium-5', 'Age-5', 'AST-5', 'Total bilirubin-5', 'TS-5', 'Hgb-5', 'Lactate-5', 'Total protein-5', 'WBC Count-5', '